In [1]:
import urllib

import mdtraj as md
import nglview
import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit
from openmm import unit as openmm_unit

from openff.interchange.components.interchange import Interchange
from openff.interchange.drivers import get_amber_energies, get_openmm_energies
from openff.interchange.drivers.all import get_summary_data

In [2]:
class _Molecule(Molecule):
    @classmethod
    def from_pdb(self, file_path):
        molecule = Molecule.from_pdb(file_path)
        molecule._add_default_hierarchy_schemes()
        molecule.perceive_hierarchy()
        return molecule


class _Topology(Topology):
    @property
    def positions(self):
        return openmm_unit.angstrom * np.vstack(
            [
                mol.conformers[0].value_in_unit(openmm_unit.angstrom)
                for mol in self.molecules
            ]
        )


def get_packed_coordinates(structure: str, n_waters: int):
    from openff.evaluator import unit as evaluator_unit
    from openff.evaluator.utils.packmol import pack_box

    water = Molecule.from_smiles("O")
    water.generate_conformers(n_conformers=1)

    trj = md.load(structure)

    if trj.unitcell_lengths:
        box_size = 1.2 * trj.unitcell_lengths
    else:
        box_size = 1.2 * (np.max(trj.xyz, axis=1) - np.min(trj.xyz, axis=1))[0]

    packed_trj, _ = pack_box(
        molecules=[water],
        number_of_copies=[n_waters],
        structure_to_solvate=structure,
        box_size=box_size * evaluator_unit.nanometer,
        # mass_density=0.8 * evaluator_unit.Unit("g/cm**3"),
    )

    return packed_trj.xyz[0]

This example uses sample data from [Protein Ligand Benchmark](https://github.com/openforcefield/protein-ligand-benchmark#proteinligandbenchmarks) data set curated by the Open Force Field Initiative. Specifially, [MCL1](https://github.com/openforcefield/protein-ligand-benchmark/tree/8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample) data is used. Conveniently for the purposes of this example, the ligand is already docked and the protein is relatively small (~2000 atoms). Follow the links for details or to swap out ligand(s).

In [3]:
url = (
    "https://raw.githubusercontent.com/openforcefield/protein-ligand-benchmark/"
    "8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample/"
)

urllib.request.urlretrieve(url + "/01_protein/crd/protein.pdb", "protein.pdb")
urllib.request.urlretrieve(url + "02_ligands/lig_23/crd/lig_23.sdf", "lig_23.sdf")

# These two files (`protein.pdb` and `lig_23.sdf`) should be in the local path now
!ls -lhrt

total 5496
-rw-r--r--  1 mwt  staff   152B Nov  3 14:18 README.md
-rw-r--r--  1 mwt  staff    52B Nov  3 14:35 scratch.py
-rw-r--r--  1 mwt  staff   2.8K Nov  4 09:51 prepare.py
-rw-r--r--  1 mwt  staff   567K Nov  4 11:18 out.gro
-rw-r--r--  1 mwt  staff   8.4K Nov  4 11:31 out.lmp
-rw-r--r--  1 mwt  staff   417B Nov  4 11:31 tmp.in
-rw-r--r--  1 mwt  staff   3.7K Nov  4 11:31 log.lammps
-rw-r--r--  1 mwt  staff   195K Nov  4 11:40 out.inpcrd
-rw-r--r--  1 mwt  staff    90B Nov  4 11:41 out.prmtop
-rw-r--r--  1 mwt  staff   248K Nov  4 11:43 docked.pdb
-rw-r--r--  1 mwt  staff   567K Nov  4 11:43 packed.pdb
-rw-r--r--  1 mwt  staff   567K Nov  4 11:48 out.pdb
-rw-r--r--  1 mwt  staff   192K Nov  4 11:51 sliced.pdb
-rw-r--r--  1 mwt  staff    32K Nov  4 11:52 protein_ligand.ipynb
-rw-r--r--  1 mwt  staff   193K Nov  4 11:52 protein.pdb
-rw-r--r--  1 mwt  staff   4.0K Nov  4 11:52 lig_23.sdf


The PDB file includes a few waters; the OpenFF Toolkit currently does not explicitly support parsing multi-component PDB files, so we'll use [MDTraj](https://mdtraj.org/) to parse the protein and save it to a new file.

In [4]:
protein_with_waters = md.load("protein.pdb")
protein_pdb = protein_with_waters.atom_slice(
    protein_with_waters.top.select("chainid 0")
)
protein_pdb.save("sliced.pdb")

Now, we can use the OpenFF Toolkit to load the protein and ligand from PDB and SDF files, respectively

In [5]:
%%capture
protein = _Molecule.from_pdb("sliced.pdb")
ligand = _Molecule.from_file("lig_23.sdf")

From these `Molecule` objects, we can make a `Topology` object and write it out to a PDB file for visualization

In [6]:
docked_topology = _Topology.from_molecules([protein, ligand])
docked_topology.to_file(filename="docked.pdb", positions=docked_topology.positions)

In [7]:
w = nglview.show_mdtraj(md.load("docked.pdb"))
w.add_representation(
    "spacefill",
    selection=[*range(protein.n_atoms, docked_topology.n_atoms)],
    color="green",
)
w

NGLWidget()

In [8]:
w.render_image()

Image(value=b'', width='99%')

Next, let's add an arbitrary number of to the system and visualize the result. The density here will be wrong; use your imagination to act like the right number of waters were added.

In [9]:
water = Molecule.from_smiles("O")
water.generate_conformers(n_conformers=1)
n_waters = 1000

packed_coordinates = get_packed_coordinates("docked.pdb", n_waters)

final_topology = _Topology.from_molecules([protein, ligand, *n_waters * [water]])
final_topology.to_file(
    filename="packed.pdb",
    positions=packed_coordinates * openmm_unit.nanometer,
)

In [10]:
w = nglview.show_mdtraj(md.load("packed.pdb"))
w.add_representation(
    "spacefill",
    selection=[*range(protein.n_atoms, docked_topology.n_atoms)],
    color="green",
)
w

NGLWidget()

In [11]:
w.render_image()

Image(value=b'', width='99%')

Now that we've prepared the topology of the system, we can apply force fields and generate inputs for simulation engines. Here, we'll use [OpenFF 2.0.0 "Sage"](https://openforcefield.org/community/news/general/sage2.0.0-release/) as a small molecule force field for the ligand and [OpenFF's port of Amber's ff14SB](https://github.com/openforcefield/amber-ff-porting/releases/tag/0.0.1) for the protein. Sage happens to include TIP3P parameters which we'll use for the waters. Because of some bugs/performance issues, we have to remote the improper torsions from the protein force field and constraints from both force fields for now.

In [12]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.1.offxml")
ff14sb.deregister_parameter_handler("ImproperTorsions")

sage = ForceField("openff-2.0.0.offxml")
sage.deregister_parameter_handler("Constraints")

For now, OpenFF's force field lines are not unified; in the future a self-consistent force field can describe both biopolymers and small molecules in one pass. But until then, we need to apply each force field to their respective components, generating an `Interchange` object for each, and then combine them using the `+` operator. This operatator uses custom code that attempts to handle combining the chemical topologies, physical forces, and positions; it's not haphazardly squishing the object together. (In this example, we're setting the positions on each topology before adding them together and then overwriting those positions later using the packed results. This is to get around a bug that  has not been fixed yet.) However, this is still a sharp edge and likely to produce strange behavior - please do not use it in production work!

In [13]:
protein_interchange = Interchange.from_smirnoff(ff14sb, protein.to_topology())
protein_interchange.positions = protein.conformers[0]

sage_interchange = Interchange.from_smirnoff(
    sage, _Topology.from_molecules([ligand, *n_waters * [water]])
)
sage_interchange.positions = _Topology.from_molecules(
    [ligand, *n_waters * [water]]
).positions

Starting handler: Bonds...
Finished handler: ... Bonds. Took 24.882378101348877
Starting handler: Angles...
Finished handler: ... Angles. Took 42.52188014984131
Starting handler: ProperTorsions...
Finished handler: ... ProperTorsions. Took 85.99561309814453
Starting handler: vdW...
Finished handler: ... vdW. Took 17.056888103485107
Starting handler: Electrostatics...
Finished handler: ... Electrostatics. Took 2.537796974182129
Starting handler: Bonds...
Finished handler: ... Bonds. Took 9.627703666687012
Starting handler: Angles...
Finished handler: ... Angles. Took 6.055186986923218
Starting handler: ProperTorsions...
Finished handler: ... ProperTorsions. Took 25.26918601989746
Starting handler: ImproperTorsions...
Finished handler: ... ImproperTorsions. Took 1.2911200523376465
Starting handler: vdW...
Finished handler: ... vdW. Took 6.068111896514893
Starting handler: Electrostatics...
Finished handler: ... Electrostatics. Took 11.703892946243286


In [14]:
combined_interchange = protein_interchange + sage_interchange
combined_interchange.positions = packed_coordinates * unit.nanometer
combined_interchange.box = [10, 10, 10]  # Arbitrary

/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:568: UserWarning: Iterchange object combination is experimental and likely to produce strange results. Use with caution!
  warnings.warn(


Now that we've prepared all atomic positions, applied each force field, and combined the results, we can visualize the result to verify that at least the positions and topology are not mangled:

In [15]:
combined_interchange.topology.to_file(
    filename="out.pdb",
    positions=combined_interchange.positions.m_as(unit.nanometer)
    * openmm_unit.nanometer,
)

w = nglview.show_mdtraj(md.load("out.pdb"))
w.add_representation(
    "spacefill",
    selection=[*range(protein.n_atoms, docked_topology.n_atoms)],
    color="green",
)
w

NGLWidget()

In [16]:
w.render_image()

Image(value=b'', width='99%')

Finally, we can export the final `Interchange` object to models understood by various simulation engines.

In [17]:
openmm_system = combined_interchange.to_openmm()
openmm_topology = combined_interchange.topology.to_openmm()
print(type(openmm_system), type(openmm_topology))

<class 'openmm.openmm.System'> <class 'openmm.app.topology.Topology'>


In [ ]:
combined_interchange.to_inpcrd("out.inpcrd")
combined_interchange.to_prmtop("out.prmtop")

In [ ]:
combined_interchange.to_gro("out.gro")
combined_interchange.to_top("out.top")

In [ ]:
combined_interchange.to_lammps("out.lmp")

In order to verify the accuracy of each export, we can use functions in the `drivers` module to call out to each engine to evaluate single-point energies. Under the hood, each function uses the export functions just as we did in the above cells. 

In [18]:
print(get_openmm_energies(combined_interchange))
print(get_amber_energies(combined_interchange))

Energies:

Bond:          		11279.5634765625 kJ / mol
Angle:         		5389.14599609375 kJ / mol
Torsion:       		7740.4345703125 kJ / mol
Nonbonded:     		None
vdW:           		5094139.870184809 kJ / mol
Electrostatics:		-20299.96211384167 kJ / mol

Energies:

Bond:          		2695.8863 kcal / mol
Angle:         		1288.0382 kcal / mol
Torsion:       		1850.0083 kcal / mol
Nonbonded:     		None
vdW:           		5094124.1539856 kJ / mol
Electrostatics:		-20104.4471888 kJ / mol



Note that some of these functions are not yet performant for systems of this size, so we are only evaluating the OpenMM and Amber interfaces. In the future, GROMACS and LAMMPS exports can be included above, and the function `get_summary_data` can be called on it. As a sneak peek, below is the result of calling that function on an `Interchange` that contains only the ligand. The data is presented as a Pandas DataFrame, which incldues convenient methods for summary statistics.

In [19]:
ligand_interchange = Interchange.from_smirnoff(sage, ligand.to_topology())
ligand_interchange.positions = ligand.conformers[0]
ligand_interchange.box = [10, 10, 10]

Starting handler: Bonds...
Finished handler: ... Bonds. Took 0.2434680461883545
Starting handler: Angles...
Finished handler: ... Angles. Took 0.14655685424804688
Starting handler: ProperTorsions...
Finished handler: ... ProperTorsions. Took 0.20629286766052246
Starting handler: ImproperTorsions...
Finished handler: ... ImproperTorsions. Took 0.023392915725708008
Starting handler: vdW...
Finished handler: ... vdW. Took 0.019572019577026367
Starting handler: Electrostatics...
Finished handler: ... Electrostatics. Took 0.028667211532592773


In [20]:
summary = get_summary_data(ligand_interchange)
summary

,Bond,Angle,Torsion,vdW,Electrostatics
OpenMM,25.839909,356.517242,27.319803,105.038192,21.036047
GROMACS,25.839901,356.516174,27.314379,105.040617,20.850627
LAMMPS,25.838186,356.516318,27.330620,104.971429,21.153593
Amber,25.837874,356.516548,27.319846,104.978234,21.009956


In [21]:
summary.describe()

,Bond,Angle,Torsion,vdW,Electrostatics
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,25.838967,356.516571,27.321162,105.007118,21.012556
std,0.001090,0.000474,0.006808,0.037398,0.124728
min,25.837874,356.516174,27.314379,104.971429,20.850627
25%,25.838108,356.516282,27.318447,104.976533,20.970124
50%,25.839043,356.516433,27.319825,105.008213,21.023001
75%,25.839903,356.516722,27.322540,105.038798,21.065433
max,25.839909,356.517242,27.330620,105.040617,21.153593
